In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from visibility_graph import visibility_graph

In [2]:
shop_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/release_10_23_2020.csv')
shop_df.head()

,session_id_hash,event_type,product_action,product_skus_hash,server_timestamp_epoch_ms,hashed_url
0,00000005c19449b57d8d05dfc8b5474be0654032,pageview,NaN,NaN,1544469996111,da99729886aff70a02733b6cd69ee7df35622d9302347e...
1,00006a0ada94a5186163a25e9ed9c94481c820d9,pageview,NaN,NaN,1545737959865,e2f7e0cee4272e804f0d323a3513dd01716a5a40ab9abf...
2,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,NaN,NaN,1544908835402,ea7b2493be61ff454f8cce412f9dc281e605daec8c43b5...
3,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,NaN,NaN,1544908847263,ea7b2493be61ff454f8cce412f9dc281e605daec8c43b5...
4,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,detail,90add4dd072e160034e98ddea257e0b59441eae00d8955...,1544909035879,8fa1ecf31ececb27ebe9c529966f3d1f907542fe138d5d...


In [3]:
print(shop_df.event_type.value_counts())
print(shop_df.product_action.value_counts())

event_type
pageview    4565253
event        868358
Name: count, dtype: int64
product_action
detail      1640190
add          743363
click         69831
remove        51512
purchase       9926
Name: count, dtype: int64


In [4]:
print(shop_df['product_action'].unique())
shop_df.drop(columns=['event_type', 'product_skus_hash', 'hashed_url'], axis=1, inplace=True)
shop_df.head()

[nan 'detail' 'click' 'add' 'purchase' 'remove']


,session_id_hash,product_action,server_timestamp_epoch_ms
0,00000005c19449b57d8d05dfc8b5474be0654032,NaN,1544469996111
1,00006a0ada94a5186163a25e9ed9c94481c820d9,NaN,1545737959865
2,00007d15aeb741b3cdd873cb3933351d699cc320,NaN,1544908835402
3,00007d15aeb741b3cdd873cb3933351d699cc320,NaN,1544908847263
4,00007d15aeb741b3cdd873cb3933351d699cc320,detail,1544909035879


In [5]:
shop_df['product_action'].replace(np.nan,1, inplace=True)
symbols = {
    'detail': 2,
    'add': 3,
    'remove': 4,
    'purchase': 5,
    'click': 6
}
shop_df['product_action'].replace(symbols, inplace=True)

In [6]:
shop_df = shop_df.groupby(shop_df.session_id_hash).agg(list).reset_index()
print(shop_df.shape)
shop_df.head()

(443660, 3)


,session_id_hash,product_action,server_timestamp_epoch_ms
0,00000005c19449b57d8d05dfc8b5474be0654032,[1],[1544469996111]
1,00006a0ada94a5186163a25e9ed9c94481c820d9,[1],[1545737959865]
2,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]","[1544908835402, 1544908847263, 1544909035879, ..."
3,0000c49d1359c87f112285d63a9fc0d4c50f73b4,"[1, 1, 1]","[1544780629084, 1544780652666, 1544780882467]"
4,0001135282dbaf0e734345212ea6f8da9f5b66ed,[1],[1544422592886]


In [7]:
# # Truncating till 5
shop_df['conversion_class'] = shop_df['product_action'].apply(lambda x: 'C' if 5 in x else 'NC')
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x[0:x.index(5)] if 5 in x else x)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) >= 5 else np.nan)
shop_df.dropna(inplace=True)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) <= 155 else np.nan)
shop_df.dropna(inplace=True)
print(shop_df.shape)
shop_df.head()  

(203084, 4)


,session_id_hash,product_action,server_timestamp_epoch_ms,conversion_class
2,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]","[1544908835402, 1544908847263, 1544909035879, ...",NC
5,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]","[1545550515947, 1545550562911, 1545550685332, ...",NC
7,00024193b68493dc098deb8a7d074ae438c615af,"[1, 1, 2, 6, 1]","[1545035634176, 1545035642856, 1545035665242, ...",NC
13,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]","[1545133756068, 1545133781992, 1545133945223, ...",NC
14,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]","[1544880442310, 1544880482619, 1544880489778, ...",NC


In [8]:
shop_df.conversion_class.value_counts()

conversion_class
NC    194755
C       8329
Name: count, dtype: int64

In [9]:
def reduced_time(time):
    index = 0
    # avg = sum(time)/(len(time))
    for sec in time:
        if index<len(time)-1:                           # do -2 when using all events
            time[index] = time[index+1] - time[index]
        index+=1
    time.pop()
    # final_val = time.pop()
    # final_val = abs(final_val - avg)
    # time.append(final_val)
    
    
    total_time = sum(time)
    index = 0
    for sec in time:
        time[index] = sec/total_time
        index+=1
    
    return time

shop_preprocessed_df = shop_df     
shop_preprocessed_df['reduced_time'] = shop_preprocessed_df['server_timestamp_epoch_ms'].apply(lambda x: reduced_time(x))

In [10]:
shop_preprocessed_df.reset_index(inplace=True)
shop_preprocessed_df.drop(['server_timestamp_epoch_ms', 'index'], axis=1, inplace=True)
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time
0,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]",NC,"[0.023753734989946568, 0.3777366561726467, 0.0..."
1,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0..."
2,00024193b68493dc098deb8a7d074ae438c615af,"[1, 1, 2, 6, 1]",NC,"[0.2186287844441086, 0.5638506876227898, 0.000..."
3,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0...."
4,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0..."


In [11]:
# creating HVGms using product action and timestamp
shop_preprocessed_df['HVGms'] = shop_preprocessed_df.apply(lambda x: visibility_graph(x['reduced_time']), axis=1)
shop_preprocessed_df

,session_id_hash,product_action,conversion_class,reduced_time,HVGms
0,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]",NC,"[0.023753734989946568, 0.3777366561726467, 0.0...","(0, 1, 2, 3, 4, 5)"
1,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,00024193b68493dc098deb8a7d074ae438c615af,"[1, 1, 2, 6, 1]",NC,"[0.2186287844441086, 0.5638506876227898, 0.000...","(0, 1, 2, 3)"
3,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0....","(0, 1, 2, 3, 4, 5, 6)"
4,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)"
...,...,...,...,...,...
203079,ffff38a875f9ac9e1b5b856ffedc0a4138b33911,"[1, 2, 1, 4, 1, 2, 1, 2, 1, 2]",NC,"[0.8197180500525593, 0.037958882509905395, 0.0...","(0, 1, 2, 3, 4, 5, 6, 7, 8)"
203080,ffff53cb919d6e46ea7cf956124d42dda833e9a2,"[1, 1, 1, 1, 1, 1, 1, 1, 1]",NC,"[0.06460747139204663, 0.13668934718514306, 0.0...","(0, 1, 2, 3, 4, 5, 6, 7)"
203081,ffff994e0a8b2fbcef28215c9a5f3ccc564f397b,"[2, 2, 2, 1, 1, 1, 1, 1, 1, 1]",NC,"[0.11213878325838, 0.0003850216818380643, 0.00...","(0, 1, 2, 3, 4, 5, 6, 7, 8)"
203082,ffffac40fa299e3832c53a3c07d3e1b8e495ab5a,"[1, 1, 1, 2, 1, 2, 1, 1, 1]",NC,"[0.18751444517925753, 0.0038060902019922335, 0...","(0, 1, 2, 3, 4, 5, 6, 7)"


In [12]:
shop_preprocessed_df['HVGms_edges'] = shop_preprocessed_df.apply(lambda x: list(x['HVGms'].edges), axis=1)
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time,HVGms,HVGms_edges
0,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]",NC,"[0.023753734989946568, 0.3777366561726467, 0.0...","(0, 1, 2, 3, 4, 5)","[(0, 1), (1, 2), (1, 3), (2, 3), (3, 4), (3, 5..."
1,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(0, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 1..."
2,00024193b68493dc098deb8a7d074ae438c615af,"[1, 1, 2, 6, 1]",NC,"[0.2186287844441086, 0.5638506876227898, 0.000...","(0, 1, 2, 3)","[(0, 1), (1, 2), (1, 3), (2, 3)]"
3,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0....","(0, 1, 2, 3, 4, 5, 6)","[(0, 1), (0, 6), (1, 2), (1, 3), (1, 4), (1, 5..."
4,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)","[(0, 1), (0, 2), (0, 10), (1, 2), (1, 10), (2,..."


# Converting edges to patterns of order 4

In [15]:
print(shop_preprocessed_df['HVGms_edges'].iloc[0])

[(0, 1), (1, 2), (1, 3), (2, 3), (3, 4), (3, 5), (4, 5)]


In [26]:
def pattern_hvg_4(df):  
    pattern_for_all_edges = list()
    for edges in df['HVGms_edges']:
        i = 0
        j = edges[len(edges)-1][1]
        pattern = list()
        for val in edges:
            while i<=int(j/2):
                edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+3]
                # print(i)
                # print(edges_tb_used)
                if len(edges_tb_used) == 5:
                    if [(i, i+1), (i, i+3), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                        pattern.append('F')
                    elif [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                        pattern.append('E')
                
                elif len(edges_tb_used) == 4:
                    if [(i, i+1), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                        pattern.append('D')
                    elif [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                        pattern.append('C')
                    elif [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                        pattern.append('B')
                    
                elif len(edges_tb_used) == 3:
                    pattern.append('A')
                
                i+=1
                    
        pattern_for_all_edges.append(pattern)
    return pattern_for_all_edges

In [27]:
pattern_hvg_4_nodes = pattern_hvg_4(shop_preprocessed_df)
shop_preprocessed_df['pattern_hvg_4_nodes'] = pattern_hvg_4_nodes

In [28]:
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time,HVGms,HVGms_edges,pattern_hvg_4_nodes
0,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]",NC,"[0.023753734989946568, 0.3777366561726467, 0.0...","(0, 1, 2, 3, 4, 5)","[(0, 1), (1, 2), (1, 3), (2, 3), (3, 4), (3, 5...","[D, C, D]"
1,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(0, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 1...","[D, C, D, C, D]"
2,00024193b68493dc098deb8a7d074ae438c615af,"[1, 1, 2, 6, 1]",NC,"[0.2186287844441086, 0.5638506876227898, 0.000...","(0, 1, 2, 3)","[(0, 1), (1, 2), (1, 3), (2, 3)]","[D, A]"
3,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0....","(0, 1, 2, 3, 4, 5, 6)","[(0, 1), (0, 6), (1, 2), (1, 3), (1, 4), (1, 5...","[D, E, D]"
4,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)","[(0, 1), (0, 2), (0, 10), (1, 2), (1, 10), (2,...","[C, D, E, F, C, D]"


In [29]:
shop_preprocessed_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/shopper_data_with_hvgms_edges_pattern_4.csv', index=False)